In [7]:
pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 56.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 101.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.3/277.3 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2

In [ ]:
pip install wandb

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import pandas as pd
from pandasql import sqldf
import matplotlib.pyplot as plt
import wandb
from sklearn.model_selection import train_test_split
from simpletransformers.question_answering import QuestionAnsweringModel
import nltk
from rouge import Rouge
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction

# Load dataset
dataset_path = "/content/train.csv"
df = pd.read_csv(dataset_path)




,qtype,Question,Answer
0,susceptibility,Who is at risk for Lymphocytic Choriomeningiti...,LCMV infections can occur after exposure to fr...
1,symptoms,What are the symptoms of Lymphocytic Choriomen...,LCMV is most commonly recognized as causing ne...
2,susceptibility,Who is at risk for Lymphocytic Choriomeningiti...,Individuals of all ages who come into contact ...
3,exams and tests,How to diagnose Lymphocytic Choriomeningitis (...,"During the first phase of the disease, the mos..."
4,treatment,What are the treatments for Lymphocytic Chorio...,"Aseptic meningitis, encephalitis, or meningoen..."


In [ ]:
def dataframe_structuring(df):
    
    data = []

    for idx, row in df.iterrows():
        qas = [{
            "id": str(idx),
            # "is_impossible": False,  
            "question": row['Question'],
            "answers": [
                {"text": row['Answer'],
                 "answer_start": 0
                }  
                ]
        }]

        data.append({
            "context": row['Answer'],  # Assuming answer as context
            "qas": qas
        })
    return data

In [ ]:
structured_data =dataframe_structuring(df)

In [ ]:
# Split dataset into training and testing sets
train_df, test_df = train_test_split(structured_data, test_size=0.25, random_state=42)


train_df.head()

In [3]:
# train_df = train_df.head(50)
# print(train_df)

                 qtype                                           Question  \
9173       information  What is (are) McLeod neuroacanthocytosis syndr...   
6723       information              What is (are) Muckle-Wells syndrome ?   
2992       information      What is (are) Chronic Eosinophilic Leukemia ?   
8700   genetic changes  What are the genetic changes related to ovaria...   
1511   exams and tests  How to diagnose Urinary Tract Infection In Adu...   
14145      information                 What is (are) Huntington disease ?   
6220   genetic changes  What are the genetic changes related to famili...   
1513        prevention  How to prevent Urinary Tract Infection In Adul...   
7852         treatment  What are the treatments for cytochrome c oxida...   
11671      information  What is (are) Neuronal ceroid lipofuscinosis 10 ?   
2596            stages  What are the stages of Childhood Brain Stem Gl...   
573           research  what research (or clinical trials) is being do...   

## BERT MODEL

In [ ]:
from simpletransformers.question_answering import QuestionAnsweringModel

# bert_args = {
#     "overwrite_output_dir": True,
#     "max_seq_length": 512,
#     "num_train_epochs": 3,
#     "train_batch_size": 8,
#     "learning_rate": 3e-5,
#     "doc_stride": 128,
#     "reprocess_input_data": True,
#     "evaluate_during_training": True,
#     "evaluate_during_training_steps": 1000,
#     "evaluate_during_training_verbose": True,
#     "use_cached_eval_features": False,
#     "save_eval_checkpoints": False,
#     "wandb_project": "medquad-bert",
#     "use_cuda": True  # Set use_cuda to False
# }

# bert_model = QuestionAnsweringModel("bert", "bert-base-uncased", args=bert_args)
# bert_model.train_model(train_data, eval_data=test_data)

wandb.init(project='NLP_A5')
# Define hyperparameters
train_args = {
    'learning_rate': 5e-5,          # Learning rate for optimizer
    'num_train_epochs': 1,          # Number of epochs for training
    'train_batch_size': 16,         # Batch size for training
    'overwrite_output_dir': True,   # Overwrite output directory if it exists
    'reprocess_input_data': True,   # Preprocess input data before training
    'evaluate_during_training': True,  # Evaluate model during training
    'wandb_project': 'NLP_A5',         # Wandb project name for logging (set to None if not using Wandb)
    'save_steps': -1,               # Save model checkpoint every X steps (-1 to disable)
    'n_best_size': 5,
    'dropout': 0.3
}

# Initialize BERT model
model = QuestionAnsweringModel('bert', 'bert-base-cased', args=train_args, use_cuda=True)

# Fine-tune the model on the training data
model.train_model(train_df, eval_data=test_df)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

...

Running Evaluation:   0%|          | 0/56 [00:00<?, ?it/s]

Running Epoch 3 of 3:   0%|          | 0/10 [00:00<?, ?it/s]


convert squad examples to features: 100%|██████████| 4102/4102 [00:51<00:00, 78.93it/s]

add example index and unique id: 100%|██████████| 4102/4102 [00:00<00:00, 254930.95it/s]


Running Evaluation:   0%|          | 0/56 [00:00<?, ?it/s]

(30,
 {'global_step': [10, 20, 30],
  'correct': [611, 568, 630],
  'similar': [3092, 3503, 3425],
  'incorrect': [399, 31, 47],
  'train_loss': [3.68408203125, 1.7044677734375, 1.02886962890625],
  'eval_loss': [-0.8920724051339286,
   -2.2418387276785716,
   -2.7223423549107144]})

# Evaluate the model on the testing data
result1, texts = model.eval_model(test_df)

# Print evaluation result
print(result1)

In [ ]:
def convert_to_predict_format(df):
    to_predict = []
    for index, row in df.iterrows():
        context = row['Answer']
        qas = [{
            "question": row['Question'],
            "id": str(index),
        }]
        to_predict.append({"context": context, "qas": qas})
    return to_predict

random_rows = df.sample(n=10)
data_prediction = convert_to_predict_format(random_rows)

In [ ]:
# function to make a list of dictionaries of actual and predicted sentences for display purposes
def show_actual_pred(model, data_prediction, actual_df):
  show_results = []
  for i in range(10):
    actual_qs = data_prediction[i]['qas'][0]['question']
    actual_ans = actual_df.iloc[i,2]
    answers =  model.predict([data_prediction[i]])
    #print(max(answers[0][0]['answer']))
    pred_ans = str(max(answers[0][0]['answer']))
    show_results.append({'actual_question':actual_qs, 'actual_answer':actual_ans ,'predicted_answer':pred_ans})
  return show_results
actual_pred = show_actual_pred(model, data_prediction, random_rows)

In [ ]:
actual_pred_df = pd.DataFrame(actual_pred)
actual_pred_df.head(10)

Bleu score

In [ ]:
def get_reference_and_predicted_answers(test_data, formatted_test_data, model):
    reference_answers = []
    predicted_answers = []

    for i in range(len(test_data)):
      actual_ans = test_data.iloc[i,2]
      answers =  model.predict([formatted_test_data[i]])
      pred_ans = str(max(answers[0][0]['answer']))
      reference_answers.append(actual_ans)
      predicted_answers.append(pred_ans)

    return reference_answers, predicted_answers
formatted_test_data = convert_to_predict_format(test_df[:100])    
ref_answers, pred_answers = get_reference_and_predicted_answers(test_df[:100], formatted_test_data, model)

In [ ]:
# Calculate BLEU score
bleu_score1 = nltk.translate.bleu_score.corpus_bleu(ref_answers, pred_answers)
print("BLEU Score:", bleu_score1)

In [ ]:
# Calculate BLEU score with smoothing function
smoothing_function = SmoothingFunction().method2  # You can choose different methods
bleu_score = corpus_bleu(ref_answers, pred_answers, smoothing_function=smoothing_function)

print("BLEU Score with Smoothing:", bleu_score)

In [ ]:
# Initialize Rouge
rouge = Rouge()

# Calculate ROUGE score
rouge_score1 = rouge.get_scores(ref_answers, pred_answers, avg=True, ignore_empty=True)
print("ROUGE Score:", rouge_score1)

## MobileBert

In [ ]:
# Define hyperparameters
train_args = {
    'learning_rate': 5e-5,          # Learning rate for optimizer
    'num_train_epochs': 1,          # Number of epochs for training
    'train_batch_size': 16,         # Batch size for training
    'overwrite_output_dir': True,   # Overwrite output directory if it exists
    'reprocess_input_data': True,   # Preprocess input data before training
    'evaluate_during_training': True,  # Evaluate model during training
    'wandb_project': 'NLP_A5',         # Wandb project name for logging (set to None if not using Wandb)
    'save_steps': -1,               # Save model checkpoint every X steps (-1 to disable)
    'n_best_size': 5,
    'dropout': 0.3
}

# Initialize BERT model
mobileBERT_model = QuestionAnsweringModel('mobilebert', 'google/mobilebert-uncased', args=train_args, use_cuda=True)

# Fine-tune the model on the training data
mobileBERT_model.train_model(train_df, eval_data = test_df)

Model Evaluation

In [ ]:
# Evaluate the model on the testing data
result_mobileBERT_1, texts = mobileBERT_model.eval_model(test_df)

# Print evaluation result
print(result_mobileBERT_1)

In [ ]:
# Plotting
plt.bar(result_mobileBERT_1.keys(), result_mobileBERT_1.values())
plt.xlabel('Evaluation Metrics')
plt.ylabel('Counts')
plt.title('Evaluation Results')
plt.show()

In [ ]:
actual_pred = show_actual_pred(mobileBERT_model, data_prediction, random_rows)

In [ ]:
actual_pred_df = pd.DataFrame(actual_pred)
actual_pred_df.head(10)

Bleu score

In [ ]:
ref_answers, pred_answers = get_reference_and_predicted_answers(test_df[:100], formatted_test_data, mobileBERT_model)

In [ ]:
# Calculate BLEU score
bleu_score = nltk.translate.bleu_score.corpus_bleu(ref_answers, pred_answers)
print("BLEU Score:", bleu_score)

In [ ]:
# Calculate BLEU score with smoothing function
smoothing_function = SmoothingFunction().method2  # You can choose different methods
bleu_score = corpus_bleu(ref_answers, pred_answers, smoothing_function=smoothing_function)

print("BLEU Score with Smoothing:", bleu_score)

In [ ]:
# Calculate ROUGE score
rouge_score = rouge.get_scores(ref_answers, pred_answers, avg=True, ignore_empty=True)
print("ROUGE Score:", rouge_score)

## RoBERTa

In [ ]:
short_formatted_data = formatted_data[:1000]
short_train_data, short_test_data = train_test_split(short_formatted_data, test_size=0.25, random_state=42)

In [ ]:
# Define hyperparameters
train_args = {
    'learning_rate': 5e-5,          # Learning rate for optimizer
    'num_train_epochs': 1,          # Number of epochs for training
    'train_batch_size': 16,         # Batch size for training
    'overwrite_output_dir': True,   # Overwrite output directory if it exists
    'reprocess_input_data': True,   # Preprocess input data before training
    'evaluate_during_training': True,  # Evaluate model during training
    'wandb_project': 'NLP_A5',         # Wandb project name for logging (set to None if not using Wandb)
    'save_steps': -1,               # Save model checkpoint every X steps (-1 to disable)
    'n_best_size': 5,
    'dropout': 0.3
}

# Initialize model
roberta_model_1 = QuestionAnsweringModel('roberta', 'roberta-base', args=train_args, use_cuda=True)

# Fine-tune the model on the training data
roberta_model_1.train_model(short_train_data, eval_data = short_test_data)

### BERT_MODEL_EVALUATION

In [10]:
# Evaluate the model on the testing data
result_roberta, texts = roberta_model_1.eval_model(short_test_data)

# Print evaluation result
print(result_roberta)

add example index and unique id: 100%|██████████| 4102/4102 [00:00<00:00, 589455.77it/s]


Running Evaluation:   0%|          | 0/56 [00:00<?, ?it/s]

In [ ]:
# Plotting
plt.bar(result_roberta.keys(), result_roberta.values())
plt.xlabel('Evaluation Metrics')
plt.ylabel('Counts')
plt.title('Evaluation Results')
plt.show()

In [ ]:
actual_pred = show_actual_pred(roberta_model_1, data_prediction, random_rows)

In [ ]:
actual_pred_df = pd.DataFrame(actual_pred)
actual_pred_df.head(10)


Computing Bleu and Rough Scores

In [ ]:
ref_answers, pred_answers = get_reference_and_predicted_answers(test_df[:100], formatted_test_data, roberta_model_1)

In [ ]:
# Calculate BLEU score
bleu_score = nltk.translate.bleu_score.corpus_bleu(ref_answers, pred_answers)
print("BLEU Score:", bleu_score)

In [ ]:
# Calculate BLEU score with smoothing function
smoothing_function = SmoothingFunction().method2  # You can choose different methods
bleu_score = corpus_bleu(ref_answers, pred_answers, smoothing_function=smoothing_function)

print("BLEU Score with Smoothing:", bleu_score)

In [ ]:
# Calculate ROUGE score
rouge_score = rouge.get_scores(ref_answers, pred_answers, avg=True, ignore_empty=True)
print("ROUGE Score:", rouge_score)

---------------------------------------------------------------------------------------------------------
MODEL END

## WANDB

In [ ]:
import wandb

wandb.login()

wandb.init(project="medquad-bert")


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


## BLEU

In [41]:
pip install Rouge rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=b9ef4e4a225eb0c82439a8f8e057c832cb5d22f6ed0089ff87ea150889768826
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [34]:
from nltk.translate.bleu_score import corpus_bleu
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction
from rouge import Rouge

# Calculate BLEU Score
def calculate_bleu_scores(actual, predicted):
    references = [[actual]]
    candidates = [predicted]
    return corpus_bleu(references, candidates)

# Calculate Rouge Score
def calculate_rouge_scores(actual, predicted):
    rouge = Rouge()
    scores = rouge.get_scores(predicted, actual)
    return scores


In [38]:
question = "What is the capital of France?"
context = "Paris is the capital of France."

to_predict = [{"context": context, "qas": [{"question": question, "id": "1"}]}]

predictions = model.predict(to_predict)

print(predictions)

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 1074.64it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

([{'id': '1', 'answer': ['', 'is the capital of France.', 'France.', 'Paris is the capital of France.', 'is the', 'Paris is the', 'is the capital', 'the capital of France.', 'Paris is the capital', 'is the capital of', 'the', 'Paris is the capital of', 'the capital', 'is', 'of France.', 'Paris is', 'the capital of', 'of']}], [{'id': '1', 'probability': [0.3021697827841778, 0.04720131117040849, 0.04539305733603502, 0.04498485907326746, 0.04459666033883892, 0.04250251594152237, 0.04189724240685956, 0.04143187770817039, 0.0399298557285139, 0.039676137991142175, 0.03914559429675259, 0.037813048660012415, 0.036776127202136646, 0.03676266184784682, 0.03513488882651295, 0.03503638185839963, 0.03482650918841561, 0.029533431653019547]}])


In [40]:
from nltk.translate.bleu_score import corpus_bleu
from rouge_score import rouge_scorer

def calculate_bleu_scores(reference_df, predictions):
    # Prepare reference answers in the correct format
    references = reference_df['answers'].tolist()
    references = [[ref] for ref in references]

    # Prepare predicted answers in the correct format
    hypotheses = [pred['answer'] for pred in predictions]

    # Calculate BLEU scores
    bleu_score = corpus_bleu(references, hypotheses)
    return bleu_score

def calculate_rouge_scores(reference_df, predictions):
    # Initialize ROUGE scorer
    rouge = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)

    # Prepare reference answers and predicted answers
    references = reference_df['answers'].tolist()
    hypotheses = [pred['answer'] for pred in predictions]

    # Calculate ROUGE scores
    rouge_scores = rouge.score(hypotheses, references)
    return rouge_scores

# Example usage:
bleu_score = calculate_bleu_scores(test_df, predictions)
rouge_scores = calculate_rouge_scores(test_df, predictions)

print("BLEU Score:", bleu_score)
print("ROUGE Scores:", rouge_scores)


ModuleNotFoundError: No module named 'rouge_score'